# Fine-tune Qwen 2.5 0.5B for Code Documentation

**Goal:** Compare with Llama 3.2 1B to select best model for UI

**Why Qwen 0.5B:**
- Smaller (0.5B vs 1B parameters)
- Faster training (~20 mins vs ~40 mins)
- Faster CPU inference for Gradio
- Strong at coding tasks

**Strategy:** Use the **same hyperparameters** that worked for Llama 1B (no tuning needed)

## Setup

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ["WANDB_DISABLED"] = "true"

Mounted at /content/drive


## Load Qwen 2.5 0.5B Model

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

print("Loading Qwen 2.5 0.5B...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-0.5B",  # Qwen model
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("✓ Qwen 2.5 0.5B loaded")
print(f"\nModel size: ~0.5B parameters (half of Llama 1B)")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

Loading Qwen 2.5 0.5B...
==((====))==  Unsloth 2025.11.6: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

✓ Qwen 2.5 0.5B loaded

Model size: ~0.5B parameters (half of Llama 1B)


## Add LoRA Adapters

**Note on Hyperparameters:**

We use **default/standard hyperparameters** (r=16, lr=2e-4, alpha=16) rather than tuning them.

**Justification:** In Llama 1B's grid search, the improvement from hyperparameter tuning was minimal (~0.4% between best and worst configs). This is likely because:
- Small sample size for grid search (1k examples, 60 steps)
- Not enough data to meaningfully differentiate configurations

Given this negligible improvement, we avoid wasting time on hyperparameter tuning for Qwen and use standard configurations that work well for small models.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,              # Best from Llama 1B
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

print("✓ LoRA adapters added with Llama 1B's best config")

Unsloth 2025.11.6 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


✓ LoRA adapters added with Llama 1B's best config


## Load Code Documentation Dataset

Same dataset as Llama 1B

In [ ]:
from datasets import load_dataset

print("Loading code documentation dataset...")
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train")

print(f"Dataset size: {len(dataset)}")
print("\nExample:")
print(f"Instruction: {dataset[0]['instruction'][:150]}...")

Loading code documentation dataset...


README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

data/train-00000-of-00001-8b6e212f3e1ece(…):   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

Dataset size: 18612

Example:
Instruction: Create a function to calculate the sum of a sequence of integers....


## Format Dataset

In [ ]:
from unsloth.chat_templates import get_chat_template

# Qwen uses chatml format
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",  # Different from Llama!
)

def format_code_documentation(example):
    instruction = example['instruction']
    output = example['output']

    if 'input' in example and example['input']:
        instruction = f"{instruction}\n\n{example['input']}"

    conversation = [
        {"role": "user", "content": instruction},
        {"role": "assistant", "content": output}
    ]

    text = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=False)
    return {"text": text}

dataset = dataset.map(format_code_documentation, remove_columns=dataset.column_names)

print("✓ Dataset formatted for Qwen (chatml)")

Unsloth: Will map <|im_end|> to EOS = <|endoftext|>.


Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

✓ Dataset formatted for Qwen (chatml)


## Train/Val/Test Split

In [ ]:
train_test = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test['train']
temp = train_test['test']
val_test = temp.train_test_split(test_size=0.5, seed=42)
val_dataset = val_test['train']
test_dataset = val_test['test']

print(f"Train: {len(train_dataset)}")
print(f"Val:   {len(val_dataset)}")
print(f"Test:  {len(test_dataset)}")

Train: 14889
Val:   1861
Test:  1862


## Training Configuration

Same settings as Llama 1B

In [ ]:
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

output_dir = "/content/drive/MyDrive/lab2_models/qwen_code_doc_model"

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=1000,
    learning_rate=2e-4,  # Best from Llama 1B
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir=output_dir,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

print("✓ Trainer configured")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/14889 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1861 [00:00<?, ? examples/s]

✓ Trainer configured


## Train

In [ ]:
print("="*80)
print("TRAINING QWEN 2.5 0.5B")
print("="*80)
print(f"\nModel: Qwen 2.5 0.5B")
print(f"Dataset: {len(train_dataset)} code examples")
print(f"Expected time: ~15-20 minutes (faster than Llama 1B!)\n")

trainer_stats = trainer.train()

print("\n" + "="*80)
print("TRAINING COMPLETE")
print("="*80)

The model is already on multiple devices. Skipping the move to device specified in `args`.


TRAINING QWEN 2.5 0.5B

Model: Qwen 2.5 0.5B
Dataset: 14889 code examples
Expected time: ~15-20 minutes (faster than Llama 1B!)



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14,889 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 8,798,208 of 502,830,976 (1.75% trained)


Step,Training Loss,Validation Loss
200,0.887600,0.856292
400,0.763100,0.846543
600,0.829400,0.839651
800,0.817900,0.835816
1000,0.915000,0.833170


Unsloth: Will smartly offload gradients to save VRAM!


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for inst


TRAINING COMPLETE


## Save Model

In [ ]:
print("Saving Qwen model...")

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✓ Model saved to: {output_dir}")

Saving Qwen model...
✓ Model saved to: /content/drive/MyDrive/lab2_models/qwen_code_doc_model


## Quantitative Comparison: Qwen vs Llama 1B

Compare both models on the same test set

In [ ]:
print("="*80)
print("COMPARING: QWEN 0.5B vs LLAMA 1B")
print("="*80)

from torch.utils.data import DataLoader

# Prepare test data
test_sample = test_dataset.select(range(min(100, len(test_dataset))))

def tokenize_for_eval(examples):
    result = tokenizer(examples["text"], truncation=True, max_length=max_seq_length)
    result["labels"] = result["input_ids"].copy()
    return result

test_tokenized = test_sample.map(tokenize_for_eval, batched=True, remove_columns=["text"])
test_dataloader = DataLoader(test_tokenized, batch_size=4, collate_fn=DataCollatorForSeq2Seq(tokenizer=tokenizer))

# Evaluate Qwen
print("\n1. Evaluating Qwen 2.5 0.5B...")
FastLanguageModel.for_inference(model)
model.eval()

total_loss = 0
num_batches = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to('cuda')
        labels = batch['labels'].to('cuda')
        outputs = model(input_ids=input_ids, labels=labels)
        total_loss += outputs.loss.item()
        num_batches += 1

qwen_loss = total_loss / num_batches
qwen_perplexity = torch.exp(torch.tensor(qwen_loss))

print(f"   Loss: {qwen_loss:.4f}")
print(f"   Perplexity: {qwen_perplexity:.2f}")

# Load and evaluate Llama 1B
print("\n2. Loading Llama 3.2 1B code model...")
llama_model, llama_tokenizer = FastLanguageModel.from_pretrained(
    model_name="/content/drive/MyDrive/lab2_models/code_doc_model/code_doc_model",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(llama_model)
llama_model.eval()
llama_tokenizer = get_chat_template(llama_tokenizer, chat_template="llama-3.1")

# Re-tokenize with Llama tokenizer
def tokenize_llama(examples):
    result = llama_tokenizer(examples["text"], truncation=True, max_length=max_seq_length)
    result["labels"] = result["input_ids"].copy()
    return result

# Use original test_dataset (before Qwen tokenization)
test_sample_llama = test_dataset.select(range(min(100, len(test_dataset))))
test_tokenized_llama = test_sample_llama.map(tokenize_llama, batched=True, remove_columns=["text"])
test_dataloader_llama = DataLoader(test_tokenized_llama, batch_size=4, collate_fn=DataCollatorForSeq2Seq(tokenizer=llama_tokenizer))

print("   Evaluating Llama 1B...")
total_loss = 0
num_batches = 0

with torch.no_grad():
    for batch in test_dataloader_llama:
        input_ids = batch['input_ids'].to('cuda')
        labels = batch['labels'].to('cuda')
        outputs = llama_model(input_ids=input_ids, labels=labels)
        total_loss += outputs.loss.item()
        num_batches += 1

llama_loss = total_loss / num_batches
llama_perplexity = torch.exp(torch.tensor(llama_loss))

print(f"   Loss: {llama_loss:.4f}")
print(f"   Perplexity: {llama_perplexity:.2f}")

# Results
print("\n" + "="*80)
print("RESULTS")
print("="*80)
print(f"\nQwen 2.5 0.5B:    Loss = {qwen_loss:.4f}, Perplexity = {qwen_perplexity:.2f}")
print(f"Llama 3.2 1B:     Loss = {llama_loss:.4f}, Perplexity = {llama_perplexity:.2f}")

print("\n" + "="*80)
print("ANALYSIS")
print("="*80)

if llama_loss < qwen_loss:
    diff = ((qwen_loss - llama_loss) / qwen_loss) * 100
    print(f"\n✓ Llama 1B is {diff:.1f}% better (more accurate)")
    print(f"\n🏆 WINNER: Llama 3.2 1B")
    print(f"\nReason: Better accuracy is more important than speed for code explanations.")
    print(f"Llama 1B will be used for the Gradio UI.")
else:
    diff = ((llama_loss - qwen_loss) / llama_loss) * 100
    print(f"\n✓ Qwen 0.5B is {diff:.1f}% better!")
    print(f"\n🏆 WINNER: Qwen 2.5 0.5B")
    print(f"\nReason: Qwen is both more accurate AND faster.")
    print(f"Qwen 0.5B will be used for the Gradio UI.")

print("\n" + "="*80)
print("TRADE-OFFS")
print("="*80)
print(f"\nQwen 0.5B Advantages:")
print(f"  • Faster training (~20 min vs ~40 min)")
print(f"  • Faster CPU inference (2x speed)")
print(f"  • Smaller GGUF file (~300 MB vs ~600 MB)")
print(f"\nLlama 1B Advantages:")
print(f"  • More parameters = potentially better quality")
print(f"  • Part of Llama family (well-supported)")
print(f"  • Better documented")

COMPARING: QWEN 0.5B vs LLAMA 1B


Map:   0%|          | 0/100 [00:00<?, ? examples/s]


1. Evaluating Qwen 2.5 0.5B...
   Loss: 0.8121
   Perplexity: 2.25

2. Loading Llama 3.2 1B code model...
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

   Evaluating Llama 1B...
   Loss: 2.1475
   Perplexity: 8.56

RESULTS

Qwen 2.5 0.5B:    Loss = 0.8121, Perplexity = 2.25
Llama 3.2 1B:     Loss = 2.1475, Perplexity = 8.56

ANALYSIS

✓ Qwen 0.5B is 62.2% better!

🏆 WINNER: Qwen 2.5 0.5B

Reason: Qwen is both more accurate AND faster.
Qwen 0.5B will be used for the Gradio UI.

TRADE-OFFS

Qwen 0.5B Advantages:
  • Faster training (~20 min vs ~40 min)
  • Faster CPU inference (2x speed)
  • Smaller GGUF file (~300 MB vs ~600 MB)

Llama 1B Advantages:
  • More parameters = potentially better quality
  • Part of Llama family (well-supported)
  • Better documented


## Summary

### What We Did:
1. ✅ Trained Qwen 2.5 0.5B on code documentation
2. ✅ Used Llama 1B's best hyperparameters (no tuning needed)
3. ✅ Compared quantitatively with Llama 1B
4. ✅ Selected winner for Gradio UI

### Why This Comparison Matters:
- **Lab requirement:** "Try multiple foundation models"
- **Practical:** Choose best for CPU inference
- **Efficient:** Reused hyperparameters, only 20 min training

### Next Steps:
1. Upload winning model to HuggingFace
2. Convert to GGUF
3. Build Gradio UI